### Building input structures for Packmol using NAGL charges

In [ ]:
from openff.interchange import Interchange
from openff.toolkit import Molecule, ForceField, Topology
import numpy as np
import mdtraj
from openff.units import unit, Quantity
from openff.toolkit.utils.nagl_wrapper import NAGLToolkitWrapper

In [ ]:
# Create a molecule from a SMILES string
# POPC is used in this example
# If missing stereochemistry, "allow_undefined_stereo=True"
lipid = Molecule.from_smiles("[C@](COP(=O)([O-])OCC[N+](C)(C)C)([H])(OC(CCCCCCC/C=C\CCCCCCCC)=O)COC(CCCCCCCCCCCCCCC)=O", allow_undefined_stereo=True)
lipid.visualize()

In [ ]:
# Verify your NAGL import
from openff.nagl_models import list_available_nagl_models
list_available_nagl_models()

In [ ]:
# Define ff version | refer to this repo for updates and alternatives for water versions: https://github.com/openforcefield/openff-forcefields
forcefield = ForceField("openff-2.2.0.offxml")

lipid.assign_partial_charges("openff-gnn-am1bcc-0.1.0-rc.3.pt", toolkit_registry=NAGLToolkitWrapper())
lipid.partial_charges

# Lipid abbreviation used for your topology and analysis
lipid.name = "POPC"

# Create Interchange topology
lipid.name = "POPC"
for i, atom in enumerate(lipid.atoms, 0):
    atom.metadata["residue_name"] = "POPC"
lipid.generate_unique_atom_names()
lipid.generate_conformers()
topology = Topology.from_molecules([lipid])
forcefield = ForceField("openff-2.2.0.offxml")
interchange = Interchange.from_smirnoff(
    force_field=forcefield,
    topology=topology,
    charge_from_molecules = [lipid]
)
interchange

# GROMACS output structure for Packmol
interchange.to_top(f"Inter_POPC.top")
interchange.to_gro(f"Inter_POPC.gro")

In [ ]:
# Get your water structure
water = Molecule.from_smiles("O")
water.visualize()

In [ ]:
# Define ff version
forcefield = ForceField("tip3p.offxml")

water.name = "TIP3P"

for i, atom in enumerate(water.atoms, 3):
    atom.metadata["residue_name"] = "TIP3P"
water.generate_unique_atom_names()
water.generate_conformers()

interchange = forcefield.create_interchange(water.to_topology())

# PDB structure for Packmol
interchange.to_pdb("water.pdb")
interchange.to_top("water.top")

### Pull the lipid for better packing

#### Build an index file with an atom from the headgroup

In [ ]:
# The nitrogen from POPC is used here since it is the furthest core atom
$ gmx make_ndx -f Inter_POPC.gro -o N1x.ndx

# Input for this index (#10 is the nitrogen in the PDB)
$ a 10
$ name 3 N1x

#### Run the pulling simulation

In [ ]:
$ gmx grompp -f pull_nvt.mdp -c Inter_POPC.gro -p Inter_POPC.top -n N1x.ndx -o pull.tpr -maxwarn 1
$ gmx mdrun -deffnm pull

# We can ignore the warning here as we are not concerned with absolute coordinate fragments for this structure

#### Convert the GRO output to a PDB

In [ ]:
$ gmx editconf -f pull.gro -o POPC.pdb

# We suggest modifing constraints in Packmol for better packing, but if you want to try a different lipid structure from the pulling instead:
$ gmx trjconv -f pull.xtc -s pull.tpr -o POPC.pdb -dump 73
# Where '73' is the best frame

########################################################################################################################################
# If these steps did not produce an ideal structure, retry with different atoms, pull in a different direction, or use multiple groups #
########################################################################################################################################

#### Run packmol with the lipid and water PDBs

In [ ]:
# If packmol is not installed, create a Conda environment
$ conda create -n bilayer
$ conda activate bilayer
$ conda install -c conda-forge packmol

###########################################################################################
# This next step can be finnicky. In 'packmol-POPC.inp,' adjust atom restraints           #
# to match your lipid. These can be found adjacent to 'atoms'. Choose two carbon          #
# atoms from the end of each tail and an atom from the headgroup (typically the nitrogen) #
###########################################################################################
$ packmol < packmol-POPC.inp

### Parametrize the system from Packmol

In [ ]:
# Define ff versions
forcefield = ForceField("openff-2.2.0.offxml", "tip3p.offxml")
lipid.assign_partial_charges("openff-gnn-am1bcc-0.1.0-rc.3.pt", toolkit_registry=NAGLToolkitWrapper())
lipid.partial_charges

# Atom count
topology = Topology.from_molecules(5120 * [water] + 128 * [lipid])

# Packmol bilayer to parametrize
path = mdtraj.load('bilayer.pdb')

# Topology designation given Packmol coordinates
topology.set_positions(path.xyz[0] * unit.nanometer)
topology.box_vectors = [7.5,7.5,9.0] * unit.nanometer

water.name = "TIP3P"
lipid.name = "POPC"

for i, atom in enumerate(lipid.atoms, 3):
    atom.metadata["residue_name"] = "POPC"
    lipid.generate_conformers()
    lipid.generate_unique_atom_names() # the default suffix for this functon is "x". Using "" will remove suffixes if desired
for i, atom in enumerate(water.atoms, 3):
    atom.metadata["residue_name"] = "TIP3P"
    water.generate_conformers()
    water.generate_unique_atom_names(suffix = "w")

interchange = forcefield.create_interchange(topology)

interchange.to_gromacs(prefix = "bilayer", decimal = 3, hydrogen_mass = 3)

# Save before running

### Run the system in GROMACS

#### Energy minimization

In [ ]:
$ gmx grompp -f min.mdp -p topol.top -c bilayer.gro -o min.tpr
$ gmx mdrun -deffnm min

#### NVT equilibration

In [ ]:
$ gmx grompp -f nvt.mdp -p topol.top -c min.gro -o nvt.tpr
$ gmx mdrun -deffnm nvt

#### NPT equilibration

In [ ]:
$ gmx grompp -f npt.mdp -p topol.top -c nvt.gro -o npt.tpr -maxwarn 1
$ gmx mdrun -deffnm npt

#### Production

In [ ]:
$ gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md.tpr
$ gmx mdrun -deffnm md

# Checkpoint run for completion
$ gmx mdrun -deffnm md -cpi md.cpt